In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By

# Automatically install the compatible version of ChromeDriver
chromedriver_autoinstaller.install()

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run in headless mode if needed

# Initialize WebDriver with updated ChromeDriver
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_autoinstaller

# Automatically download the compatible ChromeDriver for your installed version of Chrome
chromedriver_autoinstaller.install()

# Load the Excel file
excel_path = '/Users/johnquinlan/Liga_Arg_2025.xlsx'
df = pd.read_excel(excel_path)

# Clean column names to strip any leading or trailing spaces
df.columns = df.columns.str.strip()

# No row limit – scrape all rows
# Ensure the "Match Report" column is correctly named in the Excel file
if 'Match Report' not in df.columns:
    raise ValueError("The column 'Match Report' is missing from the Excel file.")

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run in headless mode

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)

# Field selectors for each item to scrape
selectors = {
    "home_record_at_time_of_match": "#content > div.scorebox > div:nth-child(1) > div:nth-child(3)",
    "away_record_at_time_of_match": "#content > div.scorebox > div:nth-child(2) > div:nth-child(3)",
    "home_possession": "#team_stats > table > tbody > tr:nth-child(3) > td:nth-child(1) > div > div:nth-child(1) > strong",
    "away_possession": "#team_stats > table > tbody > tr:nth-child(3) > td:nth-child(2) > div > div:nth-child(1) > strong",
    "home_passing_accuracy": "#team_stats > table > tbody > tr:nth-child(5) > td:nth-child(1) > div > div:nth-child(1)",
    "away_passing_accuracy": "#team_stats > table > tbody > tr:nth-child(5) > td:nth-child(2) > div > div:nth-child(1)",
    "home_shots_on_target": "#team_stats > table > tbody > tr:nth-child(7) > td:nth-child(1) > div > div:nth-child(1)",
    "away_shots_on_target": "#team_stats > table > tbody > tr:nth-child(7) > td:nth-child(2) > div > div:nth-child(1)",
    "home_saves": "#team_stats > table > tbody > tr:nth-child(9) > td:nth-child(1) > div > div:nth-child(1)",
    "away_saves": "#team_stats > table > tbody > tr:nth-child(9) > td:nth-child(2) > div > div:nth-child(1)",
    "home_fouls": "#team_stats_extra > div:nth-child(1) > div:nth-child(4)",
    "away_fouls": "#team_stats_extra > div:nth-child(1) > div:nth-child(6)",
    "home_corners": "#team_stats_extra > div:nth-child(1) > div:nth-child(7)",
    "away_corners": "#team_stats_extra > div:nth-child(1) > div:nth-child(9)",
    "home_crosses": "#team_stats_extra > div:nth-child(1) > div:nth-child(10)",
    "away_crosses": "#team_stats_extra > div:nth-child(1) > div:nth-child(12)",
    "home_touches": "#team_stats_extra > div:nth-child(1) > div:nth-child(13)",
    "away_touches": "#team_stats_extra > div:nth-child(1) > div:nth-child(15)",
    "home_tackles": "#team_stats_extra > div:nth-child(2) > div:nth-child(4)",
    "away_tackles": "#team_stats_extra > div:nth-child(2) > div:nth-child(6)",
    "home_interceptions": "#team_stats_extra > div:nth-child(2) > div:nth-child(7)",
    "away_interceptions": "#team_stats_extra > div:nth-child(2) > div:nth-child(9)",
    "home_aerials_won": "#team_stats_extra > div:nth-child(2) > div:nth-child(10)",
    "away_aerials_won": "#team_stats_extra > div:nth-child(2) > div:nth-child(12)",
    "home_clearances": "#team_stats_extra > div:nth-child(2) > div:nth-child(13)",
    "away_clearances": "#team_stats_extra > div:nth-child(2) > div:nth-child(15)",
    "home_offsides": "#team_stats_extra > div:nth-child(3) > div:nth-child(4)",
    "away_offsides": "#team_stats_extra > div:nth-child(3) > div:nth-child(6)",
    "home_goal_kicks": "#team_stats_extra > div:nth-child(3) > div:nth-child(7)",
    "away_goal_kicks": "#team_stats_extra > div:nth-child(3) > div:nth-child(9)",
    "home_throw_ins": "#team_stats_extra > div:nth-child(3) > div:nth-child(10)",
    "away_throw_ins": "#team_stats_extra > div:nth-child(3) > div:nth-child(12)",
    "home_long_balls": "#team_stats_extra > div:nth-child(3) > div:nth-child(13)",
    "away_long_balls": "#team_stats_extra > div:nth-child(3) > div:nth-child(15)",
    "home_formation": "#a > table > tbody > tr:nth-child(1) > th",
    "away_formation": "#b > table > tbody > tr:nth-child(1) > th",
}

# Initialize a dictionary to store scraped data for each field
data = {key: [] for key in selectors.keys()}

# Function to scrape stats from the match URL
def scrape_match_stats(url):
    driver.get(url)
    driver.implicitly_wait(10)

    scraped_data = {}
    for field, selector in selectors.items():
        try:
            element = driver.find_element(By.CSS_SELECTOR, selector)
            scraped_data[field] = element.text
        except Exception as e:
            print(f"{field} not found for URL: {url}, error: {e}")
            scraped_data[field] = None  # Store None if data is not found
    return scraped_data

# Iterate through all rows in the DataFrame
for index, row in df.iterrows():
    match_url = row['Match Report']
    print(f"Scraping data for: {match_url} (Row {index + 1})")
    
    scraped_data = scrape_match_stats(match_url)

    # Append the scraped values for each field to the lists in data
    for key, value in scraped_data.items():
        data[key].append(value)

    time.sleep(6)  # Wait to avoid rate limits

# Add the new columns to the DataFrame
for key, values in data.items():
    df[key] = values

# Save the updated DataFrame to a new Excel file
updated_excel_path = '/Users/johnquinlan/arg_soccer/Training_data_with_more_stats.xlsx'
df.to_excel(updated_excel_path, index=False)

# Close the driver
driver.quit()

print("Scraping completed and data saved to new Excel file.")


In [1]:
import pandas as pd

# Load the Excel file into a new DataFrame
file_path = '/Users/johnquinlan/arg_soccer/Training_data_with_more_stats.xlsx'
liga_arg = pd.read_excel(file_path)

# Print the first few rows of the new DataFrame to verify
print(liga_arg.head())

   Game_ID  Week  Day       Date      Time           Home  Home_xG  \
0        1     1  Fri 2024-05-10  19:00:00      Sarmiento      1.1   
1        2     1  Sat 2024-05-11  14:30:00        Huracán      1.6   
2        3     1  Sat 2024-05-11  15:30:00     Godoy Cruz      1.9   
3        4     1  Sat 2024-05-11  17:30:00  Independiente      0.4   
4        5     1  Sat 2024-05-11  19:45:00    River Plate      2.6   

   Home_Goals_Scored  Away_Goals_Scored  Away_xG  ... home_offsides  \
0                  1                  2      1.2  ...             2   
1                  3                  1      0.5  ...             5   
2                  0                  1      0.6  ...             1   
3                  1                  3      1.3  ...             1   
4                  3                  0      0.3  ...             3   

  away_offsides home_goal_kicks away_goal_kicks home_throw_ins away_throw_ins  \
0             2              13               8             29         

In [2]:
# Function to clean the record and calculate the winning percentage
def calculate_winning_percentage(record):
    # Remove any non-numeric characters except for the dash '-'
    cleaned_record = ''.join(char if char.isdigit() or char == '-' else '' for char in record)
    
    # If the cleaned record has the correct format, calculate the winning percentage
    if len(cleaned_record.split('-')) == 3:
        wins, draws, losses = map(int, cleaned_record.split('-'))
        total_games = wins + draws + losses
        return round(wins / total_games, 3) if total_games > 0 else 0
    else:
        return 0  # In case the format is still incorrect

# Extract only the formation (text inside parentheses) for home_formation and away_formation
liga_arg['home_formation'] = liga_arg['home_formation'].str.extract(r'\(([^)]+)\)')
liga_arg['away_formation'] = liga_arg['away_formation'].str.extract(r'\(([^)]+)\)')

# Extract completed and attempted passes from home_passing_accuracy
liga_arg['home_passing_accuracy'] = liga_arg['home_passing_accuracy'].astype(str)
liga_arg[['home_passes_completed', 'home_passes_attempted']] = liga_arg['home_passing_accuracy'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column home_passing_accuracy by dividing home_passes_completed by home_passes_attempted
liga_arg['home_passing_accuracy'] = (liga_arg['home_passes_completed'] / liga_arg['home_passes_attempted']).round(2)

# Extract completed and attempted passes from away_passing_accuracy
liga_arg['away_passing_accuracy'] = liga_arg['away_passing_accuracy'].astype(str)
liga_arg[['away_passes_completed', 'away_passes_attempted']] = liga_arg['away_passing_accuracy'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column away_passing_accuracy by dividing away_passes_completed by away_passes_attempted
liga_arg['away_passing_accuracy'] = (liga_arg['away_passes_completed'] / liga_arg['away_passes_attempted']).round(2)

# Extract shots on target and total shots from home_shots_on_target
liga_arg['home_shots_on_target'] = liga_arg['home_shots_on_target'].astype(str)
liga_arg[['home_shots_on_target', 'home_shots_attempted']] = liga_arg['home_shots_on_target'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column home_shots_on_target_percentage
liga_arg['home_shots_on_target_percentage'] = (liga_arg['home_shots_on_target'] / liga_arg['home_shots_attempted'] * 100).round(2)

# Extract shots on target and total shots from away_shots_on_target
liga_arg['away_shots_on_target'] = liga_arg['away_shots_on_target'].astype(str)
liga_arg[['away_shots_on_target', 'away_shots_attempted']] = liga_arg['away_shots_on_target'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column away_shots_on_target_percentage
liga_arg['away_shots_on_target_percentage'] = (liga_arg['away_shots_on_target'] / liga_arg['away_shots_attempted'] * 100).round(2)

# Extract home saves (shots saved) and home saves attempts (shots on target)
liga_arg['home_saves'] = liga_arg['home_saves'].astype(str)
liga_arg[['home_saves_completed', 'home_saves_attempted']] = liga_arg['home_saves'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column home_save_percentage by dividing home_saves_completed by home_saves_attempted and converting to percentage
liga_arg['home_save_percentage'] = (liga_arg['home_saves_completed'] / liga_arg['home_saves_attempted'] * 100).round(2)

# Extract away saves (shots saved) and away saves attempts (shots on target)
liga_arg['away_saves'] = liga_arg['away_saves'].astype(str)
liga_arg[['away_saves_completed', 'away_saves_attempted']] = liga_arg['away_saves'].str.extract(r'(\d+) of (\d+)').astype(float)

# Create a new column away_save_percentage by dividing away_saves_completed by away_saves_attempted and converting to percentage
liga_arg['away_save_percentage'] = (liga_arg['away_saves_completed'] / liga_arg['away_saves_attempted'] * 100).round(2)


# Create Home_Match_Outcome column based on home_goals_scored vs away_goals_scored
liga_arg['Home_Match_Outcome'] = liga_arg.apply(
    lambda row: 'W' if row['Home_Goals_Scored'] > row['Away_Goals_Scored'] else 
                ('L' if row['Home_Goals_Scored'] < row['Away_Goals_Scored'] else 'D'), axis=1
)

# Create Away_Match_Outcome column based on home_goals_scored vs away_goals_scored
liga_arg['Away_Match_Outcome'] = liga_arg.apply(
    lambda row: 'W' if row['Away_Goals_Scored'] > row['Home_Goals_Scored'] else 
                ('L' if row['Away_Goals_Scored'] < row['Home_Goals_Scored'] else 'D'), axis=1
)

# Apply the winning percentage calculation to the home and away records at the time of match
liga_arg['home_winning_percentage_at_time_of_match'] = liga_arg['home_record_at_time_of_match'].apply(calculate_winning_percentage)
liga_arg['away_winning_percentage_at_time_of_match'] = liga_arg['away_record_at_time_of_match'].apply(calculate_winning_percentage)


# Print the updated DataFrame
print(liga_arg.head())




   Game_ID  Week  Day       Date      Time           Home  Home_xG  \
0        1     1  Fri 2024-05-10  19:00:00      Sarmiento      1.1   
1        2     1  Sat 2024-05-11  14:30:00        Huracán      1.6   
2        3     1  Sat 2024-05-11  15:30:00     Godoy Cruz      1.9   
3        4     1  Sat 2024-05-11  17:30:00  Independiente      0.4   
4        5     1  Sat 2024-05-11  19:45:00    River Plate      2.6   

   Home_Goals_Scored  Away_Goals_Scored  Away_xG  ... home_saves_completed  \
0                  1                  2      1.2  ...                  1.0   
1                  3                  1      0.5  ...                  3.0   
2                  0                  1      0.6  ...                  3.0   
3                  1                  3      1.3  ...                  2.0   
4                  3                  0      0.3  ...                  2.0   

  home_saves_attempted home_save_percentage away_saves_completed  \
0                  3.0                33.3

In [3]:
liga_arg = liga_arg.drop(columns=['home_saves', 'away_saves'])


In [ ]:
import pandas as pd

# Identify all columns in the DataFrame
columns = liga_arg.columns

# Identify Home and Away columns
home_columns = [col for col in columns if col.startswith("Home_")]
away_columns = [col for col in columns if col.startswith("Away_")]

# Create a mapping of new column names
generic_columns = [col.replace("Home_", "") for col in home_columns]

# Initialize an empty list to store the new rows
new_rows = []

# Iterate over every row in the original DataFrame
for _, row in liga_arg.iterrows():
    # Create the Home row
    home_row = {
        "game_id": row["Game_ID"],
        "Team": row["Home"],
        "Opponent": row["Away"],
        "Home_Away": "Home",
    }
    for home_col, generic_col in zip(home_columns, generic_columns):
        home_row[generic_col] = row[home_col]
    
    # Create the Away row
    away_row = {
        "game_id": row["Game_ID"],
        "Team": row["Away"],
        "Opponent": row["Home"],
        "Home_Away": "Away",
    }
    for away_col, generic_col in zip(away_columns, generic_columns):
        away_row[generic_col] = row[away_col]
    
    # Append the rows to the list
    new_rows.append(home_row)
    new_rows.append(away_row)

# Convert the new rows into a DataFrame
liga_arg_transformed = pd.DataFrame(new_rows)

# Display the transformed DataFrame
print(liga_arg_transformed)


In [4]:
liga_arg.columns = liga_arg.columns.str.lower()


print(list(liga_arg.columns))

['game_id', 'week', 'day', 'date', 'time', 'home', 'home_xg', 'home_goals_scored', 'away_goals_scored', 'away_xg', 'away', 'venue', 'match report', 'home_record_at_time_of_match', 'away_record_at_time_of_match', 'home_possession', 'away_possession', 'home_passing_accuracy', 'away_passing_accuracy', 'home_shots_on_target', 'away_shots_on_target', 'home_fouls', 'away_fouls', 'home_corners', 'away_corners', 'home_crosses', 'away_crosses', 'home_touches', 'away_touches', 'home_tackles', 'away_tackles', 'home_interceptions', 'away_interceptions', 'home_aerials_won', 'away_aerials_won', 'home_clearances', 'away_clearances', 'home_offsides', 'away_offsides', 'home_goal_kicks', 'away_goal_kicks', 'home_throw_ins', 'away_throw_ins', 'home_long_balls', 'away_long_balls', 'home_formation', 'away_formation', 'home_passes_completed', 'home_passes_attempted', 'away_passes_completed', 'away_passes_attempted', 'home_shots_attempted', 'home_shots_on_target_percentage', 'away_shots_attempted', 'away_sho

In [5]:
# Define the mapping for Home and Away prefixes
# Strip spaces from column names
liga_arg.columns = liga_arg.columns.str.strip()

# Ensure unique column names (in case of duplicates)
liga_arg.columns = pd.Index([
    f'{col}_{i}' if col == f'{col}_{i-1}' else col for i, col in enumerate(liga_arg.columns)
])

# Define the list of required columns for the final DataFrame
required_columns = [
    'week', 'day', 'date', 'time', 'home', 'home_xg', 'home_goals_scored',
    'away_goals_scored', 'away_xg', 'away', 'venue', 'match report',
    'home_record_at_time_of_match', 'away_record_at_time_of_match',
    'home_possession', 'away_possession', 'home_passing_accuracy',
    'away_passing_accuracy', 'home_shots_on_target', 'away_shots_on_target',
    'home_fouls', 'away_fouls', 'home_corners', 'away_corners',
    'home_crosses', 'away_crosses', 'home_touches', 'away_touches',
    'home_tackles', 'away_tackles', 'home_interceptions',
    'away_interceptions', 'home_aerials_won', 'away_aerials_won',
    'home_clearances', 'away_clearances', 'home_offsides', 'away_offsides',
    'home_goal_kicks', 'away_goal_kicks', 'home_throw_ins',
    'away_throw_ins', 'home_long_balls', 'away_long_balls',
    'home_formation', 'away_formation', 'home_passes_completed',
    'home_passes_attempted', 'away_passes_completed',
    'away_passes_attempted', 'home_shots_attempted',
    'home_shots_on_target_percentage', 'away_shots_attempted',
    'away_shots_on_target_percentage', 'home_saves_completed',
    'home_saves_attempted', 'home_save_percentage', 'away_saves_completed',
    'away_saves_attempted', 'away_save_percentage', 'home_match_outcome',
    'away_match_outcome', 'team', 'opponent', 'home_away', 'xg',
    'goals_scored', 'record_at_time_of_match', 'possession',
    'passing_accuracy', 'shots_on_target', 'fouls', 'corners', 'crosses',
    'touches', 'tackles', 'interceptions', 'aerials_won', 'clearances',
    'offsides', 'goal_kicks', 'throw_ins', 'long_balls', 'formation',
    'passes_completed', 'passes_attempted', 'shots_attempted',
    'shots_on_target_percentage', 'saves_completed', 'saves_attempted',
    'save_percentage', 'match_outcome'
]

# Add missing columns with default values
for col in required_columns:
    if col not in liga_arg.columns:
        liga_arg[col] = None

# Transform data into a format where each row represents one team's data
final_rows = []
for index, row in liga_arg.iterrows():
    # Home team data
    home_row = row.copy()
    home_row['team'] = row['home']
    home_row['opponent'] = row['away']
    home_row['home_away'] = 'home'
    home_row['xg'] = row['home_xg']
    home_row['goals_scored'] = row['home_goals_scored']
    home_row['record_at_time_of_match'] = row['home_record_at_time_of_match']
    home_row['possession'] = row['home_possession']
    home_row['passing_accuracy'] = row['home_passing_accuracy']
    home_row['shots_on_target'] = row['home_shots_on_target']
    home_row['fouls'] = row['home_fouls']
    home_row['corners'] = row['home_corners']
    home_row['crosses'] = row['home_crosses']
    home_row['touches'] = row['home_touches']
    home_row['tackles'] = row['home_tackles']
    home_row['interceptions'] = row['home_interceptions']
    home_row['aerials_won'] = row['home_aerials_won']
    home_row['clearances'] = row['home_clearances']
    home_row['offsides'] = row['home_offsides']
    home_row['goal_kicks'] = row['home_goal_kicks']
    home_row['throw_ins'] = row['home_throw_ins']
    home_row['long_balls'] = row['home_long_balls']
    home_row['formation'] = row['home_formation']
    home_row['passes_completed'] = row['home_passes_completed']
    home_row['passes_attempted'] = row['home_passes_attempted']
    home_row['shots_attempted'] = row['home_shots_attempted']
    home_row['shots_on_target_percentage'] = row['home_shots_on_target_percentage']
    home_row['saves_completed'] = row['home_saves_completed']
    home_row['saves_attempted'] = row['home_saves_attempted']
    home_row['save_percentage'] = row['home_save_percentage']
    home_row['match_outcome'] = row['home_match_outcome']
    home_row['winning_percentage'] = row['home_winning_percentage_at_time_of_match']
    final_rows.append(home_row)

    # Away team data
    away_row = row.copy()
    away_row['team'] = row['away']
    away_row['opponent'] = row['home']
    away_row['home_away'] = 'away'
    away_row['xg'] = row['away_xg']
    away_row['Goals_Scored'] = row['away_goals_scored']
    away_row['record_at_time_of_match'] = row['away_record_at_time_of_match']
    away_row['possession'] = row['away_possession']
    away_row['passing_accuracy'] = row['away_passing_accuracy']
    away_row['shots_on_target'] = row['away_shots_on_target']
    away_row['fouls'] = row['away_fouls']
    away_row['corners'] = row['away_corners']
    away_row['crosses'] = row['away_crosses']
    away_row['touches'] = row['away_touches']
    away_row['tackles'] = row['away_tackles']
    away_row['interceptions'] = row['away_interceptions']
    away_row['aerials_won'] = row['away_aerials_won']
    away_row['clearances'] = row['away_clearances']
    away_row['offsides'] = row['away_offsides']
    away_row['goal_kicks'] = row['away_goal_kicks']
    away_row['throw_ins'] = row['away_throw_ins']
    away_row['long_balls'] = row['away_long_balls']
    away_row['formation'] = row['away_formation']
    away_row['passes_completed'] = row['away_passes_completed']
    away_row['passes_attempted'] = row['away_passes_attempted']
    away_row['shots_attempted'] = row['away_shots_attempted']
    away_row['shots_on_target_percentage'] = row['away_shots_on_target_percentage']
    away_row['saves_completed'] = row['away_saves_completed']
    away_row['saves_attempted'] = row['away_saves_attempted']
    away_row['save_percentage'] = row['away_save_percentage']
    away_row['match_outcome'] = row['away_match_outcome']
    away_row['winning_percentage'] = row['away_winning_percentage_at_time_of_match']
    final_rows.append(away_row)

# Create the final DataFrame
liga_transform = pd.DataFrame(final_rows)
import pandas as pd

# Strip spaces from column names
liga_arg.columns = liga_arg.columns.str.strip()

# Ensure unique column names (in case of duplicates)
liga_arg.columns = pd.Index([
    f'{col}_{i}' if col == f'{col}_{i-1}' else col for i, col in enumerate(liga_arg.columns)
])

# Define the required columns for the final DataFrame
required_columns = [
    'week', 'day', 'date', 'time', 'home', 'home_xg', 'home_goals_scored',
    'away_goals_scored', 'away_xg', 'away', 'venue', 'match report',
    'home_record_at_time_of_match', 'away_record_at_time_of_match',
    'home_possession', 'away_possession', 'home_passing_accuracy',
    'away_passing_accuracy', 'home_shots_on_target', 'away_shots_on_target',
    'home_fouls', 'away_fouls', 'home_corners', 'away_corners',
    'home_crosses', 'away_crosses', 'home_touches', 'away_touches',
    'home_tackles', 'away_tackles', 'home_interceptions',
    'away_interceptions', 'home_aerials_won', 'away_aerials_won',
    'home_clearances', 'away_clearances', 'home_offsides', 'away_offsides',
    'home_goal_kicks', 'away_goal_kicks', 'home_throw_ins',
    'away_throw_ins', 'home_long_balls', 'away_long_balls',
    'home_formation', 'away_formation', 'home_passes_completed',
    'home_passes_attempted', 'away_passes_completed',
    'away_passes_attempted', 'home_shots_attempted',
    'home_shots_on_target_percentage', 'away_shots_attempted',
    'away_shots_on_target_percentage', 'home_saves_completed',
    'home_saves_attempted', 'home_save_percentage', 'away_saves_completed',
    'away_saves_attempted', 'away_save_percentage', 'home_match_outcome',
    'away_match_outcome', 'team', 'opponent', 'home_away', 'xG',
    'goals_scored', 'record_at_time_of_match', 'possession',
    'passing_accuracy', 'shots_on_target', 'fouls', 'corners', 'crosses',
    'touches', 'tackles', 'interceptions', 'aerials_won', 'clearances',
    'offsides', 'goal_kicks', 'throw_ins', 'long_balls', 'formation',
    'passes_completed', 'passes_attempted', 'shots_attempted',
    'shots_on_target_percentage', 'saves_completed', 'saves_attempted',
    'save_percentage', 'winning_percentage', 'match_outcome'
]

# Add missing columns with default values
for col in required_columns:
    if col not in liga_arg.columns:
        liga_arg[col] = None

# Transform data into a format where each row represents one team's data
final_rows = []

for index, row in liga_arg.iterrows():
    for home_away, prefix in {'home': 'home', 'away': 'away'}.items():
        team_row = row.copy()
        team_row['team'] = row[f'{home_away}']
        team_row['opponent'] = row['away'] if home_away == 'home' else row['home']
        team_row['home_away'] = home_away
        team_row['xg'] = row[f'{prefix}_xg']
        team_row['goals_scored'] = row[f'{prefix}_goals_scored']
        team_row['record_at_time_of_match'] = row[f'{prefix}_record_at_time_of_match']
        team_row['possession'] = row[f'{prefix}_possession']
        team_row['passing_accuracy'] = row[f'{prefix}_passing_accuracy']
        team_row['shots_on_target'] = row[f'{prefix}_shots_on_target']
        team_row['fouls'] = row[f'{prefix}_fouls']
        team_row['corners'] = row[f'{prefix}_corners']
        team_row['crosses'] = row[f'{prefix}_crosses']
        team_row['touches'] = row[f'{prefix}_touches']
        team_row['tackles'] = row[f'{prefix}_tackles']
        team_row['interceptions'] = row[f'{prefix}_interceptions']
        team_row['aerials_won'] = row[f'{prefix}_aerials_won']
        team_row['clearances'] = row[f'{prefix}_clearances']
        team_row['offsides'] = row[f'{prefix}_offsides']
        team_row['goal_kicks'] = row[f'{prefix}_goal_kicks']
        team_row['throw_ins'] = row[f'{prefix}_throw_ins']
        team_row['long_balls'] = row[f'{prefix}_long_balls']
        team_row['formation'] = row[f'{prefix}_formation']
        team_row['passes_completed'] = row[f'{prefix}_passes_completed']
        team_row['passes_attempted'] = row[f'{prefix}_passes_attempted']
        team_row['shots_attempted'] = row[f'{prefix}_shots_attempted']
        team_row['shots_on_target_percentage'] = row[f'{prefix}_shots_on_target_percentage']
        team_row['saves_completed'] = row[f'{prefix}_saves_completed']
        team_row['saves_attempted'] = row[f'{prefix}_saves_attempted']
        team_row['save_percentage'] = row[f'{prefix}_save_percentage']
        team_row['match_outcome'] = row[f'{home_away}_match_outcome']
        team_row['winning_percentage'] = row[f'{prefix}_winning_percentage_at_time_of_match']
        final_rows.append(team_row)

# Create the final DataFrame
liga_transform = pd.DataFrame(final_rows)

# Reset index to avoid duplicates
liga_transform.reset_index(drop=True, inplace=True)

# Display the first few rows of the transformed DataFrame
print(liga_transform.head())


   game_id  week  day       date      time        home  home_xg  \
0        1     1  Fri 2024-05-10  19:00:00   Sarmiento      1.1   
1        1     1  Fri 2024-05-10  19:00:00   Sarmiento      1.1   
2        2     1  Sat 2024-05-11  14:30:00     Huracán      1.6   
3        2     1  Sat 2024-05-11  14:30:00     Huracán      1.6   
4        3     1  Sat 2024-05-11  15:30:00  Godoy Cruz      1.9   

   home_goals_scored  away_goals_scored  away_xg  ... passes_completed  \
0                  1                  2      1.2  ...            175.0   
1                  1                  2      1.2  ...            327.0   
2                  3                  1      0.5  ...            329.0   
3                  3                  1      0.5  ...            368.0   
4                  0                  1      0.6  ...            322.0   

  passes_attempted shots_attempted shots_on_target_percentage saves_completed  \
0            313.0            16.0                      31.25          

In [ ]:
# Check for duplicates in the index of final_rows
print(pd.Index(final_rows).duplicated().any())


In [ ]:
print(liga_transform)

In [6]:
# List of columns to remove
columns_to_remove = [
    "home", "home_xg", "home_goals_scored", "away_goals_scored", "away_xg", "away",
    "home_record_at_time_of_match", "away_record_at_time_of_match", "home_possession", "away_possession",
    "home_passing_accuracy", "away_passing_accuracy", "home_shots_on_target", "away_shots_on_target",
    "home_fouls", "away_fouls", "home_corners", "away_corners", "home_crosses", "away_crosses",
    "home_touches", "away_touches", "home_tackles", "away_tackles", "home_interceptions", "away_interceptions",
    "home_aerials_won", "away_aerials_won", "home_clearances", "away_clearances", "home_offsides", "away_offsides",
    "home_goal_kicks", "away_goal_kicks", "home_throw_ins", "away_throw_ins", "home_long_balls", "away_long_balls",
    "home_formation", "away_formation", "home_passes_completed", "home_passes_attempted", "away_passes_completed",
    "away_passes_attempted", "home_shots_attempted", "home_shots_on_target_percentage", "away_shots_attempted",
    "away_shots_on_target_percentage", "home_saves_completed", "home_saves_attempted", "home_save_percentage",
    "away_saves_completed", "away_saves_attempted", "away_save_percentage", "home_match_outcome",
    "away_match_outcome", "home_winning_percentage_at_time_of_match", "away_winning_percentage_at_time_of_match",
    "away_Goals_scored", "xG", "Goals_Scored"
]

# Drop the columns from the DataFrame
liga_transform = liga_transform.drop(columns=columns_to_remove, errors='ignore')

# Display the updated DataFrame
print(liga_transform)



     game_id  week  day       date      time  \
0          1     1  Fri 2024-05-10  19:00:00   
1          1     1  Fri 2024-05-10  19:00:00   
2          2     1  Sat 2024-05-11  14:30:00   
3          2     1  Sat 2024-05-11  14:30:00   
4          3     1  Sat 2024-05-11  15:30:00   
..       ...   ...  ...        ...       ...   
751      376    27  Sat 2024-12-14  21:15:00   
752      377    27  Mon 2024-12-16  21:00:00   
753      377    27  Mon 2024-12-16  21:00:00   
754      378    27  Mon 2024-12-16  21:00:00   
755      378    27  Mon 2024-12-16  21:00:00   

                                           venue  \
0                     Estadio Eva Peron de Junín   
1                     Estadio Eva Peron de Junín   
2                      Estadio Tomás Adolfo Ducó   
3                      Estadio Tomás Adolfo Ducó   
4                     Estadio Feliciano Gambarte   
..                                           ...   
751        Estadio Presidente Juan Domingo Perón   
752    

In [7]:
# Define columns with '%' symbol to transform
columns_with_percentage_symbol = ['possession']

# Transform columns with '%' symbol
for column in columns_with_percentage_symbol:
    liga_transform[column] = liga_transform[column] \
        .str.replace('%', '') \
        .astype(float) / 100

# Define columns without '%' symbol to transform
columns_without_percentage_symbol = ['shots_on_target_percentage', 'save_percentage']

# Transform columns without '%' symbol
for column in columns_without_percentage_symbol:
    liga_transform[column] = liga_transform[column].astype(float) / 100

# Check the updated DataFrame
print(liga_transform.head())



   game_id  week  day       date      time                       venue  \
0        1     1  Fri 2024-05-10  19:00:00  Estadio Eva Peron de Junín   
1        1     1  Fri 2024-05-10  19:00:00  Estadio Eva Peron de Junín   
2        2     1  Sat 2024-05-11  14:30:00   Estadio Tomás Adolfo Ducó   
3        2     1  Sat 2024-05-11  14:30:00   Estadio Tomás Adolfo Ducó   
4        3     1  Sat 2024-05-11  15:30:00  Estadio Feliciano Gambarte   

                                        match report            team  \
0  https://fbref.com/en/matches/4fefca20/Sarmient...       Sarmiento   
1  https://fbref.com/en/matches/4fefca20/Sarmient...       Instituto   
2  https://fbref.com/en/matches/ffacd80b/Huracan-...         Huracán   
3  https://fbref.com/en/matches/ffacd80b/Huracan-...  Defensa y Just   
4  https://fbref.com/en/matches/c22ece76/Godoy-Cr...      Godoy Cruz   

           opponent home_away  ...  formation  passes_completed  \
0         Instituto      home  ...    4-4-1-1          

In [8]:
# List of fields you want to average
fields_to_average = [
    'goals_scored', 'possession', 'passing_accuracy', 'shots_on_target', 'fouls', 
    'corners', 'crosses', 'touches', 'tackles', 'interceptions', 'aerials_won', 
    'clearances', 'offsides', 'goal_kicks', 'throw_ins', 'long_balls', 'passes_completed', 
    'passes_attempted', 'shots_attempted', 'shots_on_target_percentage', 'saves_completed', 
    'saves_attempted', 'save_percentage'
]

# Calculate lagged averages for each field
for field in fields_to_average:
    avg_field_name = f"avg_{field}"
    
    # Calculate the rolling average (expanding mean)
    liga_transform[avg_field_name] = (
        liga_transform.groupby('team')[field]  # Group by team
        .apply(lambda x: x.shift(1).expanding().mean())  # Apply the expanding mean
        .reset_index(level=0, drop=True)  # Align index with the original DataFrame
    )
    
#change record field name 
liga_transform.rename(columns={'record_at_time_of_match': 'record'}, inplace=True)


# Check the output
print(liga_transform.tail())


     game_id  week  day       date      time  \
751      376    27  Sat 2024-12-14  21:15:00   
752      377    27  Mon 2024-12-16  21:00:00   
753      377    27  Mon 2024-12-16  21:00:00   
754      378    27  Mon 2024-12-16  21:00:00   
755      378    27  Mon 2024-12-16  21:00:00   

                                           venue  \
751        Estadio Presidente Juan Domingo Perón   
752             Estadio Juan Bautista Gargantini   
753             Estadio Juan Bautista Gargantini   
754  Estadio Monumental Presidente José Fierr...   
755  Estadio Monumental Presidente José Fierr...   

                                          match report  \
751  https://fbref.com/en/matches/5e5f5389/Racing-C...   
752  https://fbref.com/en/matches/57e20023/Independ...   
753  https://fbref.com/en/matches/57e20023/Independ...   
754  https://fbref.com/en/matches/5c4a18f8/Atletico...   
755  https://fbref.com/en/matches/5c4a18f8/Atletico...   

                        team                 oppo

In [10]:
liga_transform.to_csv('/Users/johnquinlan/arg_soccer/Liga_Arg_2024_Season.csv', index=False)
